## Amazon SageMaker Distributed Processing jobs

With Amazon SageMaker Processing jobs, you can leverage a simplified, managed experience to run data pre- or post-processing and model evaluation workloads on the Amazon SageMaker platform.

A processing job downloads input from Amazon Simple Storage Service (Amazon S3), then uploads outputs to Amazon S3 during or after the processing job.

<img src="Processing-1.jpg">

This notebook shows how you can run a distributed processing job to run a scikit-learn script that cleans, pre-processes, performs feature engineering, and splits the input data into train and test sets.

The dataset used here is the [Census-Income KDD Dataset](https://archive.ics.uci.edu/ml/datasets/Census-Income+%28KDD%29). You select features from this dataset, clean the data, and turn the data into features that the training algorithm can use to train a binary classification model, and split the data into train and test sets. The task is to predict whether rows representing census responders have an income greater than `$50,000`, or less than `$50,000`. The dataset is heavily class imbalanced, with most records being labeled as earning less than `$50,000`. After training a logistic regression model, you evaluate the model against a hold-out test dataset, and save the classification evaluation metrics, including precision, recall, and F1 score for each label, and accuracy and ROC AUC for the model.

## Data pre-processing and feature engineering

To run the scikit-learn preprocessing script as a processing job, create a `SKLearnProcessor`, which lets you run scripts inside of processing jobs using the scikit-learn image provided.

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

region = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = get_execution_role()
sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.m5.xlarge',
                                     instance_count=3)

Copy the dataset into three different prefixes in the same bucket. this way, every instance of the SageMaker Processing jobs will process one file

In [ ]:
import boto3
s3 = boto3.resource('s3')
source= { 'Bucket' : 'sagemaker-sample-data-{}'.format(region), 'Key': 'processing/census/census-income.csv'}
dest = s3.Bucket(bucket)
dest.copy(source, 'sagemaker/dist-processing/census/001/census-income.csv')
dest.copy(source, 'sagemaker/dist-processing/census/002/census-income.csv')
dest.copy(source, 'sagemaker/dist-processing/census/003/census-income.csv')

Before introducing the script you use for data cleaning, pre-processing, and feature engineering, inspect the first 20 rows of the dataset in `001` prefix. The target is predicting the `income` category. The features from the dataset you select are `age`, `education`, `major industry code`, `class of worker`, `num persons worked for employer`, `capital gains`, `capital losses`, and `dividends from stocks`.

In [ ]:
import pandas as pd

prefix = "sagemaker/processing/census/"

input_folder = 's3://{}/sagemaker/dist-processing/census/'.format(bucket)
input_file = '{}001/census-income.csv'.format(input_folder)

df = pd.read_csv(input_file, nrows=10)
df.head(n=10)

This notebook cell writes a file `preprocessing.py`, which contains the pre-processing script. You can update the script, and rerun this cell to overwrite `preprocessing.py`. You run this as a processing job in the next cell. In this script, you

* Remove duplicates and rows with conflicting data
* transform the target `income` column into a column containing two labels.
* transform the `age` and `num persons worked for employer` numerical columns into categorical features by binning them
* scale the continuous `capital gains`, `capital losses`, and `dividends from stocks` so they're suitable for training
* encode the `education`, `major industry code`, `class of worker` so they're suitable for training
* split the data into training and test datasets, and saves the training features and labels and test features and labels.


In [ ]:
%%writefile preprocessing.py

import argparse
import os
import warnings
import sys

import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer, KBinsDiscretizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

INPUT_PATH = '/opt/ml/processing/input/'

columns = ['age', 'education', 'major industry code', 'class of worker', 'num persons worked for employer',
           'capital gains', 'capital losses', 'dividends from stocks', 'income']
class_labels = [' - 50000.', ' 50000+.']

s3_client = boto3.resource("s3")


def upload_objects(bucket, prefix, local_path):
    try:
        bucket_name = bucket  # s3 bucket name
        root_path = local_path  # local folder for upload

        print('bucket_name: {}'.format(bucket_name))
        print('root_path: {}'.format(root_path))
        
        s3_bucket = s3_client.Bucket(bucket_name)

        for path, subdirs, files in os.walk(root_path):
            for file in files:
                print('Uploading file: {}'.format(os.path.join(path, file)))               
                s3_bucket.upload_file(
                    os.path.join(path, file), "sagemaker/dist-processing/census/output/{}/{}".format(prefix, file)
                )
    except Exception as err:
        print(err)
        
        
def print_shape(df):
    negative_examples, positive_examples = np.bincount(df['income'])
    print('Data shape: {}, {} positive examples, {} negative examples'.format(df.shape, positive_examples, negative_examples))

if __name__=='__main__':
    args_iter = iter(sys.argv[1:])
    args = dict(zip(args_iter, args_iter))
    print('Received arguments: {}'.format(args))

    # Creating the necessary paths to save the output files
    if not os.path.exists("/opt/ml/processing/train"):
        os.makedirs("/opt/ml/processing/train")

    if not os.path.exists("/opt/ml/processing/test"):
        os.makedirs("/opt/ml/processing/test")
    
    prefix = [folder for folder in os.listdir(INPUT_PATH) if folder.startswith('0')][0]
    
    print('Received prefix: {}'.format(prefix))
    print('Files in Input Directory: {}'.format(os.listdir(INPUT_PATH + prefix)))

    input_data_path = os.path.join(INPUT_PATH + prefix, 'census-income.csv')
    
    print('Reading input data from {}'.format(input_data_path))
    df = pd.read_csv(input_data_path)
    df = pd.DataFrame(data=df, columns=columns)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    df.replace(class_labels, [0, 1], inplace=True)
    
    negative_examples, positive_examples = np.bincount(df['income'])
    print('Data after cleaning: {}, {} positive examples, {} negative examples'.format(df.shape, positive_examples, negative_examples))
    
    split_ratio = 0.2
    print('Splitting data into train and test sets with ratio {}'.format(split_ratio))
    X_train, X_test, y_train, y_test = train_test_split(df.drop('income', axis=1), df['income'], test_size=split_ratio, random_state=0)

    preprocess = make_column_transformer(
        (['age', 'num persons worked for employer'], KBinsDiscretizer(encode='onehot-dense', n_bins=10)),
        (['capital gains', 'capital losses', 'dividends from stocks'], StandardScaler()),
        (['education', 'major industry code', 'class of worker'], OneHotEncoder(sparse=False))
    )
    print('Running preprocessing and feature engineering transformations')
    train_features = preprocess.fit_transform(X_train)
    test_features = preprocess.transform(X_test)
    
    print('Train data shape after preprocessing: {}'.format(train_features.shape))
    print('Test data shape after preprocessing: {}'.format(test_features.shape))
    
    train_features_output_path = os.path.join('/opt/ml/processing/train', 'train_features.csv')
    train_labels_output_path = os.path.join('/opt/ml/processing/train', 'train_labels.csv')
    
    test_features_output_path = os.path.join('/opt/ml/processing/test', 'test_features.csv')
    test_labels_output_path = os.path.join('/opt/ml/processing/test', 'test_labels.csv')
    
    print('Saving training features to {}'.format(train_features_output_path))
    pd.DataFrame(train_features).to_csv(train_features_output_path, header=False, index=False)
    
    print('Saving test features to {}'.format(test_features_output_path))
    pd.DataFrame(test_features).to_csv(test_features_output_path, header=False, index=False)
    
    print('Saving training labels to {}'.format(train_labels_output_path))
    y_train.to_csv(train_labels_output_path, header=False, index=False)
    
    print('Saving test labels to {}'.format(test_labels_output_path))
    y_test.to_csv(test_labels_output_path, header=False, index=False)
    
    upload_objects(
        args['s3_output_bucket'],
        prefix,
        "/opt/ml/processing/train/",
    )
    upload_objects(
        args['s3_output_bucket'],
        prefix,
        "/opt/ml/processing/test/",
    )
    
    print('Processing Complete')


Run this script as a processing job. Use the `SKLearnProcessor.run()` method. You give the `run()` method one `ProcessingInput` where the `source` is the census dataset in Amazon S3, and the `destination` is where the script reads this data from, in this case `/opt/ml/processing/input`. These local paths inside the processing container must begin with `/opt/ml/processing/`.

Also give the `run()` method a `ProcessingOutput`, where the `source` is the path the script writes output data to. For outputs, the `destination` defaults to an S3 bucket that the Amazon SageMaker Python SDK creates for you, following the format `s3://sagemaker-<region>-<account_id>/<processing_job_name>/output/<output_name/`. You also give the ProcessingOutputs values for `output_name`, to make it easier to retrieve these output artifacts after the job is run.

The `arguments` parameter in the `run()` method are command-line arguments in our `preprocessing.py` script.

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(code='preprocessing.py',
                      inputs=[ProcessingInput(
                        source=input_folder,
                        s3_data_distribution_type='ShardedByS3Key',
                        destination='/opt/ml/processing/input')],
                      arguments=[                          
                          's3_output_bucket', bucket
                      ]
                     )

preprocessing_job_description = sklearn_processor.jobs[-1].describe()


Let's inspect the output files created

In [ ]:
output_folder = 's3://{}/sagemaker/dist-processing/census/output/'.format(bucket)
output_folder
! aws s3 ls $output_folder --recursive

Now inspect the output of the pre-processing job for `001` prefix, which consists of the processed features.

In [ ]:
training_features = pd.read_csv(output_folder + '001/train_features.csv'.format(bucket), nrows=10)
print('Training features shape: {}'.format(training_features.shape))
training_features.head(n=10)